In [27]:
from hera.shared import global_config
from hera.auth import ArgoCLITokenGenerator

global_config.host = "http://127.0.0.0:2746"
global_config.token = ArgoCLITokenGenerator
global_config.verify_ssl = False

## WorkflowTemplate <-> Pipeline & Workflow <-> Flow

WorkFlowTemplates crucially allow the definition of parameters as input variables to the entire workflow *without specifying a value*. You can also specify the default value, but it is possible to omit in cases where a sensible value just doesnt apply.

Submitting these WorkFlow templates doesnt execute anything on the ArgoWorkflow engine, but rather adds a template that future `Workflow`s can be derived from, together with configurable workflow input parameters.

This allows for the following entity mapping between bettmensch.ai pipelines and ArgoWorkflow resources:

| bettmensch.ai | ArgoWorkflow      |
| --------------|-------------------|
| `Flow`        | `WorkflowTemplate`|
| `Run`         | `Workflow`        |

## Define script templates

In [28]:
from hera.workflows import script, Parameter, models as m

@script(outputs=[Parameter(name="sum",value_from=m.ValueFrom(path="./sum.txt"))])
def add(a: float, b: float):
    with open('./sum.txt','w') as output:
        output.write(str(a + b))
        
@script(outputs=[Parameter(name="difference",value_from=m.ValueFrom(path="./difference.txt"))])
def subtract(a: float, b: float):
    with open('./difference.txt','w') as output:
        output.write(str(a - b))
        
@script(outputs=[Parameter(name="product",value_from=m.ValueFrom(path="./product.txt"))])
def multiply(a: float, b: float):
    with open('./product.txt','w') as output:
        output.write(str(a * b))

type(add), type(subtract), type(multiply)

(function, function, function)

## Define Workflow and DAG

In [29]:
from hera.workflows import WorkflowTemplate, DAG, Parameter, S3Artifact

dag_name = "2ab-minus-bcd"

with WorkflowTemplate(
    generate_name="2ab-minus-bcd-", 
    entrypoint=dag_name,
    namespace="argo",
    arguments=[Parameter(name="a"),Parameter(name="b"),Parameter(name="c"),Parameter(name="d")],
    # the Workflow referencing this template inherits the `spec` level `workflow_metadata` field from this WorkflowTemplate, so
    # when submitting via .create(), the workflow metadata does get merged into the Workflow CRD manifest before being applied to the K8s cluster
    workflow_metadata={
        'annotations':{'annotation_key_a':'annotation_value_a'},
        'labels':{'label_key_a':'label_value_a'}
    },
    # the Workflow referencing this template inherits the `spec` level `pod_metadata` field from this WorkflowTemplate, so
    # when submitting via .create(), the workflow metadata does get merged into the Pod manifest(s) before being applied to the K8s cluster.
    # for whatever reason, the (apparently?) pod level manifests shown on the ArgoWorkflow server dashboard do not show this
    pod_metadata={ 
        'annotations':{'annotation_key_b':'annotation_value_b'},
        'labels':{'label_key_b':'label_value_b'}
    }) as wt:
    
    wt_a, wt_b, wt_c, wt_d = wt.get_parameter('a').value,wt.get_parameter('b').value,wt.get_parameter('c').value,wt.get_parameter('d').value 
    
    with DAG(name=dag_name) as dag:
        double_a = multiply(
            name="multiply-2-and-a",arguments=[
                Parameter(name="a",value=2), 
                Parameter(name="b",value=wt_a)
            ]
        )
        bc = multiply(
            name="multiply-b-and-c",arguments=[
                Parameter(name="a",value=wt_b),
                Parameter(name="b",value=wt_c)
            ]
        )
        double_ab = multiply(
            name="multiply-2a-and-b",arguments=[
                Parameter(name="a",value=double_a.get_parameter('product').value),
                Parameter(name="b",value=wt_b)
            ]
        )
        bcd = multiply(
            name="multiply-bc-and-d",arguments=[
                Parameter(name="a",value=bc.get_parameter('product').value),
                Parameter(name="b",value=wt_d)
            ]
        )
        double_ab_minus_bcd = subtract(
            name="subtract-2ab-and-bcd",arguments=[
                Parameter(name="a",value=double_ab.get_parameter('product').value),
                Parameter(name="b",value=bcd.get_parameter('product').value)
            ],    
        )
        
        print(wt_b)
        print(bc.get_parameter('product').value)
        
        double_a >> double_ab
        bc >> bcd
        [double_ab, bcd] >> double_ab_minus_bcd       

{{workflow.parameters.b}}
{{tasks.multiply-b-and-c.outputs.parameters.product}}


In [30]:
wt.create()

ConnectionError: HTTPConnectionPool(host='127.0.0.0', port=2746): Max retries exceeded with url: /api/v1/workflow-templates/argo (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7eebce4c2560>: Failed to establish a new connection: [Errno 111] Connection refused'))